In [1]:
import numpy as np
from mpnet.sst_envs.utils import load_data, get_obs
import pickle
import time
import click
from tqdm.auto import tqdm
from pathlib import Path
import importlib
from matplotlib import pyplot as plt

import sys
sys.path.append('/media/arclabdl1/HD1/Linjun/mpc-mpnet-py/deps/sparse_rrt-1')

from sparse_rrt import _deep_smp_module

In [ ]:
planner_params = {
    'solver_type': "none",
    'system_type': "quadortor_obs",
    'start_state': np.array([0, 0, 0,
                             0, 0, 0, 0,
                             0, 0, 0,
                             0, 0, 0]),
    'goal_state': np.array([5, 5, 5, 
                            0, 0, 0, 0,
                            0, 0, 0, 
                            0, 0, 0]),
    'goal_radius': 2,
    'random_seed': 0,
    'sst_delta_near': 0.5,
    'sst_delta_drain': 0.5,
    'obs_list': np.array([[1,2,3]]),
    'width': 0,
    'verbose': False,
    'mpnet_weight_path': "mpnet/exported/output/cartpole_obs/mpnet_10k_external_small_model.pt", 
    'cost_predictor_weight_path': "mpnet/exported/output/cartpole_obs/cost_10k.pt",
    'cost_to_go_predictor_weight_path': "mpnet/exported/output/cartpole_obs/cost_to_go_obs.pt",
    'num_sample': 1,
    'weights_array': np.array([1, 1, 1,
                             0.5, 0.5, 0.5, 0.5,
                             0.2, 0.2, 0.2, 
                             0.2, 0.2, 0.2]),
    'obs_voxel_array': []
}


planner = _deep_smp_module.DSSTMPCWrapper(**planner_params)